In [1]:
import pathlib
import logging
import threading
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
np.random.seed(0)
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
print(tf.__version__)
from sklearn.model_selection import StratifiedKFold
import time
dispositivoUsado = '/cpu:0' 
#dispositivoUsado = '/gpu:0'
exibirRodando = False
QuantasThreadsFinalizaram = 0
anonimo = False
plt.rcParams['figure.dpi'] = 600

2.7.0


In [2]:
dados = pd.read_csv('dados-Ambos-Tudo.csv')
dados["QoE"] = dados["QoE"] + 3

#---------------------------------------------------------
#DROPANDO LINHAS COM USO DE CPU E RAM ALTO

dados = dados[dados['UsoCpu'] < 81]
dados = dados[dados['UsoRam'] < 81]
dados.sample(frac=1).reset_index(drop=True)

#DROPANDO LINHAS COM USO DE CPU E RAM ALTO
#---------------------------------------------------------


dados = dados.sample(frac=1).reset_index(drop=True)
#Salvando quantos jogadores temos e quantas instancias temos



dicionarioJogadores = dados['NOME'].value_counts().to_dict()

#'NOME'
dadosFear = dados[dados['Jogo'] == 'FEAR']
dadosHL2 = dados[dados['Jogo'] == 'Half-Life 2']
dados = dados[dados['Jogo'] != 'FEAR']
numeroInstancias = len(dados)
dados.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida','Jogo'], axis=1, inplace=True)
dados.dropna(inplace=True)

dadosHL2.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida','Jogo'], axis=1, inplace=True)
dadosHL2.dropna(inplace=True)

dadosFear.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida','Jogo'], axis=1, inplace=True)
dadosFear.dropna(inplace=True)
porcentagemTeste = 0.2
porcentagemValidacao = 0.1
NumeroCamadasInternas = 64
print(numeroInstancias)

2020


In [12]:
def arredondaVetor(vetor):
 with tf.device(dispositivoUsado):
    novoVetor = []
    for x in range(len(vetor)):
        if(vetor[x]>=0):
            floatTeste = vetor[x] - int(vetor[x])
            if(floatTeste>=0.5):
                novoVetor.append(int(vetor[x])+1)
            else:
                novoVetor.append(int(vetor[x]))
        else:
            valor = vetor[x]*(-1.0)
            floatTeste = valor - int(valor)
            if(floatTeste>=0.5):
                novoVetor.append((int(valor)+1)*-1)
            else:
                novoVetor.append(int(valor)*-1)
    return novoVetor

In [13]:
def porcentagemAcerto(vetor1,vetor2):
 with tf.device(dispositivoUsado):
    tamanho = float(len(vetor1))
    acertos = float(0)
    for x in range(len(vetor1)):
        if(vetor1[x]==vetor2[x]):
            acertos = acertos + 1.0
    acertos = (acertos/tamanho)*100.0
    return acertos

In [14]:
def rodarModeloRF(baseDados):
    if 'NOME' in baseDados.columns:
        baseDados.drop(['NOME'], axis=1, inplace=True)
        baseDados.dropna(inplace=True)
    n_estimators = 41
    max_features = 3
    max_depth = 7
    global porcentagemTeste
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    XT = list()
    YT = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X_train,y_train):
        rfr=RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,criterion='mse',bootstrap=True,max_depth=max_depth)
        rfr.fit(X_train[train],y_train[train])
        y_pred=rfr.predict(X_train[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y_train[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y_train[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
        XT.append(X_train[test])
        YT.append(y_train[test])
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))

    return mse,acerto,model,X_test,y_test

In [24]:
def rodarModeloRFFear(baseDados):
    if 'NOME' in baseDados.columns:
        baseDados.drop(['NOME'], axis=1, inplace=True)
        baseDados.dropna(inplace=True)
    n_estimators = 41
    max_features = 3
    max_depth = 7
    global porcentagemTeste
    global porcentagemValidacao
    
    tempFear = dadosFear.copy(deep=True)
    if 'NOME' in tempFear.columns:
        tempFear.drop(['NOME'], axis=1, inplace=True)
        tempFear.dropna(inplace=True)
    
    
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    
    XFear = np.asarray(tempFear[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    
    y = np.asarray(baseDados['QoE'])
    
    yFear = np.asarray(tempFear['QoE'])
    
    scaler = StandardScaler()
    
    scalerFear = StandardScaler()
    
    scaler.fit(X)
    
    scalerFear.fit(XFear)
    
    X = scaler.transform(X)
    
    XFear = scalerFear.transform(XFear)
    
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    
    X_tempFear = np.zeros((tempFear.shape[0],tempFear.shape[1]))
    
    X_temp[:,:X.shape[1]] = X
    
    X_tempFear[:,:XFear.shape[1]] = XFear
    
    X = X_temp
    
    XFear = X_tempFear
    
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    
    XFear[:, 6:-1] = np.asarray(tempFear[list(tempFear.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    
    XFear = np.asarray(XFear).astype(np.float32)
    yFear = np.asarray(yFear).astype(np.float32)
    
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    XT = list()
    YT = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X_train, y_train):
        rfr=RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,criterion='mse',bootstrap=True,max_depth=max_depth)
        rfr.fit(X_train[train],y_train[train])
        y_pred=rfr.predict(X_train[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y_train[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y_train[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
        XT.append(X_train[test])
        YT.append(y_train[test])
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(XFear)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(yFear,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(yFear,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,XFear,yFear

In [16]:
def plot_heatMap(test_predictions,y_test,nomeHeatmap):
    import seaborn as sns
    test_predictions_round = arredondaVetor(test_predictions)
    x_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for x-axis
    y_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for y-axis
    
    heatMatrix = np.zeros((7,7))
    for x in range(len(y_test)):
        intX = int(test_predictions_round[x])
        intY = int(y_test[x])
        if(intX > 6):
            continue
        heatMatrix[intX][intY] = heatMatrix[intX][intY] +1
                
    sns.heatmap(heatMatrix, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="Greys")
    plt.savefig(nomeHeatmap+'.png', transparent=False)
    plt.show()

In [17]:
def plot_melhor(y_test, test_predictions,history,tipo):
 with tf.device(dispositivoUsado):
    histi = pd.DataFrame(history.history)
    histi['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Absoluto [QoE]')
    plt.plot(histi['epoch'], histi['mae'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mae'],label = 'Erro de Validação')
    plt.ylim([0,5])
    plt.legend()
    plt.savefig('melhor'+tipo+'ResSemAcordoMAE.png')
    plt.show()
    
        
    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Quadratico [$QoE^2$]')
    plt.plot(histi['epoch'], histi['mse'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mse'],label = 'Erro de Validação')
    plt.ylim([0,20])
    plt.legend()
    plt.savefig('melhor'+tipo+'ResSemAcordoMSE.png')
    plt.show()
    
    
    plt.figure()
    plt.scatter(y_test, test_predictions)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig('melhor'+tipo+'ResSemAcordoPredicoesSemArredondar.png')
    plt.show()
    
    
    
    
    test_predictions_round = arredondaVetor(test_predictions)
    print("Porcentagem de acertos arredondado: "+str(porcentagemAcerto(y_test,test_predictions_round)))
    plt.figure()
    plt.scatter(y_test, test_predictions_round)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões Arredondadas [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig('melhor'+tipo+'ResSemAcordoPredicoesArredondado.png')
    plt.show()
    
    plot_heatMap(test_predictions,y_test,"HeatMap_"+tipo)

In [18]:
repeticoes = 30
mse,acerto,model,X_test,y_test = rodarModeloRF(dados)
    for ABA in range(repeticoes):
        mse1,acerto1,model1,X_test1,y_test1 = rodarModeloRFFear(dados)
        if(mse1<mse):
            mse = mse1
            acerto = acerto1
            X_test = X_test1
            y_test = y_test1
            model = model1


model.predict(X_train)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

# 